In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/emaar_sample_timeseries.csv')

In [ ]:
  df.head(2)

,Date,Close
0,2022-01-03,4.53
1,2022-01-04,4.54


In [ ]:
df.columns

Index(['Date', 'Close'], dtype='object')

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Fetch historical data
ticker = yf.Ticker("EMAAR.AE")  # Emaar Properties on DFM
df = ticker.history(period="2y")[["Close"]].dropna()

# 2. Setup supervised data
df["Target"] = df["Close"].shift(-1)
X = np.array(df["Close"][:-1]).reshape(-1, 1)
y = np.array(df["Target"][:-1])

# 3. Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,random_state=42)
# 4. Fit linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 5. Evaluate
y_pred = lr.predict(X_test)
print("LR RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# 6. Predict tomorrow
last_close = df["Close"].iloc[-1]
tomorrow_pred_lr = lr.predict([[last_close]])[0]
print("Linear Regression predicts:", tomorrow_pred_lr)


LR RMSE: 0.22064570527383326
Linear Regression predicts: 14.887435475193092


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# 1. Fetch data
df = yf.Ticker("EMAAR.AE").history(period="2y")[["Close"]].dropna()

# 2. Preprocess
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

# Create sequences: past 60 days map to the next-day price
SEQ_LEN = 60
X, y = [], []
for i in range(SEQ_LEN, len(scaled)):
    X.append(scaled[i-SEQ_LEN:i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# 3. Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,random_state=42)

# 4. Build LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error")

# 5. Train
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 6. Predict tomorrow
last_seq = scaled[-SEQ_LEN:]
X_pred = last_seq.reshape((1, SEQ_LEN, 1))
pred_scaled = model.predict(X_pred)
tomorrow_pred_lstm = scaler.inverse_transform(pred_scaled)[0, 0]
print("LSTM predicts:", tomorrow_pred_lstm)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 177ms/step - loss: 0.1607 - val_loss: 0.0489
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 0.0294 - val_loss: 0.0134
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0184 - val_loss: 0.0044
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0086 - val_loss: 0.0037
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0064 - val_loss: 0.0034
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0067 - val_loss: 0.0031
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0049 - val_loss: 0.0035
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0053 - val_loss: 0.0034
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0057 - val_loss: 0.0029
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0049 - val_loss: 0.0029
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0064 - val_loss: 0.0030
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0063 - val_loss: 0.0027
Epoch 13/20

In [ ]:
last_close = df["Close"].iloc[-1]
tomorrow_pred_lr = lr.predict([[last_close]])[0]
print("Linear Regression predicts:", tomorrow_pred_lr)


Linear Regression predicts: 14.887435475193092


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Fetch historical data
ticker = yf.Ticker("EMAAR.AE")  # Emaar Properties on DFM
df = ticker.history(period="2y")[["Close"]].dropna()

# 2. Setup supervised data
df["Target"] = df["Close"].shift(-1)
X = np.array(df["Close"][:-1]).reshape(-1, 1)
y = np.array(df["Target"][:-1])

# 3. Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,random_state=42)

# 4. Fit linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# 5. Evaluate
y_pred = lr.predict(X_test)
print("LR RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# 6. Predict for the next week (5 days)
last_close = df["Close"].iloc[-1]
next_day_pred = last_close
next_week_preds = []

# Predict for the next 5 days iteratively
for i in range(5):
    next_day_pred = lr.predict([[next_day_pred]])[0]
    next_week_preds.append(next_day_pred)
    print(f"Linear Regression predicts day {i+1}:", next_day_pred)

# The last prediction in next_week_preds is the prediction for the end of the week
nextweek_pred_lr = next_week_preds[-1]
print("\nLinear Regression predicts end of next week:", nextweek_pred_lr)

LR RMSE: 0.22064569687698488
Linear Regression predicts day 1: 14.887435474070728
Linear Regression predicts day 2: 14.924987966313074
Linear Regression predicts day 3: 14.962658226372048
Linear Regression predicts day 4: 15.00044662357756
Linear Regression predicts day 5: 15.038353528417773

Linear Regression predicts end of next week: 15.038353528417773


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# 1. Fetch data
df = yf.Ticker("EMAAR.AE").history(period="2y")[["Close"]].dropna()

# 2. Preprocess
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

# Create sequences: past 60 days map to the next-day price
SEQ_LEN = 60
X, y = [], []
for i in range(SEQ_LEN, len(scaled)):
    X.append(scaled[i-SEQ_LEN:i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# 3. Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,random_state=42)

# 4. Build LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error")

# 5. Train
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict for the next 5 days iteratively
for i in range(5):
    next_day_pred = lr.predict([[next_day_pred]])[0]
    next_week_preds.append(next_day_pred)
    print(f"Linear Regression predicts day {i+1}:", next_day_pred)

# The last prediction in next_week_preds is the prediction for the end of the week
nextweek_pred_lr = next_week_preds[-1]
print("\nLinear Regression predicts end of next week:", nextweek_pred_lr)



Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - loss: 0.1504 - val_loss: 0.0316
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.0300 - val_loss: 0.0091
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0130 - val_loss: 0.0040
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0088 - val_loss: 0.0031
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0058 - val_loss: 0.0028
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0050 - val_loss: 0.0031
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.0039 - val_loss: 0.0027
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0047 - val_loss: 0.0030
Epoch 9/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0038 - val_loss: 0.0032
Epoch 10/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0043 - val_loss: 0.0028
Epoch 11/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0050 - val_loss: 0.0031
Epoch 12/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.0051 - val_loss: 0.0031
Epoch 13/20

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split # Import train_test_split here

# 1. Fetch historical data
ticker = yf.Ticker("EMAAR.AE")  # Emaar Properties on DFM
df = ticker.history(period="2y")[["Close"]].dropna()

# 2. Setup supervised data
df["Target"] = df["Close"].shift(-1)
# Ensure there are enough samples after shifting and dropping NaNs
if len(df) > 1:
    X = np.array(df["Close"][:-1]).reshape(-1, 1)
    y = np.array(df["Target"][:-1])
else:
    print("Not enough data to train the model after processing.")
    # You might want to exit or handle this case appropriately
    X = np.array([])
    y = np.array([])


# 3. Train-test split
# Only attempt to split if there's data
if len(X) > 0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)

    # 4. Fit linear regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # 5. Evaluate (Optional - commented out for brevity if only prediction is needed)
    # y_pred = lr.predict(X_test)
    # print("LR RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

    # Now, use 'df' and 'lr' to make the prediction
    last_close = df["Close"].iloc[-1]
    nextweek_pred_lr = lr.predict([[last_close]])[0]
    print("Linear Regression predicts:", nextweek_pred_lr)
else:
    print("Linear Regression model not trained due to insufficient data.")

Linear Regression predicts: 14.887435475193092
